In [4]:
import numpy as np
import pandas as pd

pd.set_option("display.max.columns", 100)
# to draw pictures in jupyter notebook
%matplotlib inline
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [5]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
DATA_URL = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"

In [6]:
data = pd.read_csv(DATA_URL + "adult.data.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 1. How many men and women (sex feature) are represented in this dataset?

In [7]:
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

### 2. What is the average age (age feature) of women?

In [8]:
data[data.sex=="Female"]["age"].mean()

36.85823043357163

### 3. What is the percentage of German citizens (native-country feature)?

In [9]:
float((data["native-country"] == "Germany").sum()) * 100 / data.shape[0]

0.42074874850281013

In [10]:
data.shape[0]

32561

In [11]:
data["native-country"].value_counts()

United-States                 29170
Mexico                          643
?                               583
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France                      

### 4-5. What are the mean and standard deviation of age for those who earn more than 50K per year (salary feature) and those who earn less than 50K per year?

In [23]:
for tap, tap2 in data.groupby('salary'):
    print(tap, " std - ",  tap2.age.std())
    print(tap, " mean - ", tap2.age.mean())

<=50K  std -  14.020088490824813
<=50K  mean -  36.78373786407767
>50K  std -  10.51902771985177
>50K  mean -  44.24984058155847


### 6. Is it true that people who earn more than 50K have at least high school education? (education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate feature)

In [31]:
data[data.salary == ">50K"].education.unique()

array(['HS-grad', 'Masters', 'Bachelors', 'Some-college', 'Assoc-voc',
       'Doctorate', 'Prof-school', 'Assoc-acdm', '7th-8th', '12th',
       '10th', '11th', '9th', '5th-6th', '1st-4th'], dtype=object)

In [33]:
data.shape

(32561, 15)

### 7. Display age statistics for each race (race feature) and each gender (sex feature). Use groupby() and describe(). Find the maximum age of men of Amer-Indian-Eskimo race.

In [35]:
data.groupby(["race", "sex"]).age.describe()

count       mean        std   min   25%   50%  \
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [36]:
data[(data.race == "Amer-Indian-Eskimo")&(data.sex == "Male")].age.max()

82

### 8. Among whom is the proportion of those who earn a lot (>50K) greater: married or single men (marital-status feature)? Consider as married those who have a marital-status starting with Married (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.

In [55]:
def toFixed(numObj, digits=0):
    return f"{numObj:.{digits}f}"

In [49]:
RichMarMen = data[(data.sex == "Male")&(data.salary == ">50K")&((data['marital-status'] == "Married-civ-spouse")|  \
    (data['marital-status'] == "Married-spouse-absent")|(data['marital-status'] == "Married-AF-spouse"))].shape[0]


In [85]:
RichMarMen2 = data[(data["sex"] == "Male")
     & (data["marital-status"].str.startswith("Married")) & (data["salary"]==">50K")].shape[0]

In [50]:
RichMen = data[(data.sex == "Male")&(data.salary == ">50K")].shape[0]

In [52]:
RichMen

6662

In [53]:
RichMarMen

5965

In [86]:
RichMarMen2

5965

In [56]:
toFixed(RichMarMen/RichMen*100, 2)

'89.54'

In [87]:
# married men
data[(data["sex"] == "Male")
     & (data["marital-status"].str.startswith("Married"))][
    "salary"
].value_counts(normalize=True)

<=50K    0.559486
>50K     0.440514
Name: salary, dtype: float64

In [88]:
# single men
data[
    (data["sex"] == "Male")
    & ~(data["marital-status"].str.startswith("Married"))
]["salary"].value_counts(normalize=True)

<=50K    0.915505
>50K     0.084495
Name: salary, dtype: float64

### 9. What is the maximum number of hours a person works per week (hours-per-week feature)? How many people work such a number of hours, and what is the percentage of those who earn a lot (>50K) among them?



In [61]:
data["hours-per-week"].nunique(), \
data["hours-per-week"].max()

(94, 99)

In [70]:
RPeopleMaxHours = data[(data["hours-per-week"] == data["hours-per-week"].max())&(data.salary == ">50K")].shape
PeopleMaxHours = data[(data["hours-per-week"] == data["hours-per-week"].max())].shape
percentRinPeopleMaxHours = RPeopleMaxHours[0]/PeopleMaxHours[0]*100

In [90]:
RPeopleMaxHours[0], PeopleMaxHours[0]

(25, 85)

In [71]:
percentRinPeopleMaxHours

29.411764705882355

### 10. Count the average time of work (hours-per-week) for those who earn a little and a lot (salary) for each country (native-country). What will these be for Japan?

In [73]:
data.groupby(["salary","native-country"])["hours-per-week"].agg("mean")

salary  native-country 
<=50K   ?                  40.164760
        Cambodia           41.416667
        Canada             37.914634
        China              37.381818
        Columbia           38.684211
                             ...    
>50K    Thailand           58.333333
        Trinadad&Tobago    40.000000
        United-States      45.505369
        Vietnam            39.200000
        Yugoslavia         49.500000
Name: hours-per-week, Length: 82, dtype: float64

In [91]:
for (country, salary), sub_df in data.groupby(["native-country", "salary"]):
    print(country, salary, round(sub_df["hours-per-week"].mean(), 2))

? <=50K 40.16
? >50K 45.55
Cambodia <=50K 41.42
Cambodia >50K 40.0
Canada <=50K 37.91
Canada >50K 45.64
China <=50K 37.38
China >50K 38.9
Columbia <=50K 38.68
Columbia >50K 50.0
Cuba <=50K 37.99
Cuba >50K 42.44
Dominican-Republic <=50K 42.34
Dominican-Republic >50K 47.0
Ecuador <=50K 38.04
Ecuador >50K 48.75
El-Salvador <=50K 36.03
El-Salvador >50K 45.0
England <=50K 40.48
England >50K 44.53
France <=50K 41.06
France >50K 50.75
Germany <=50K 39.14
Germany >50K 44.98
Greece <=50K 41.81
Greece >50K 50.62
Guatemala <=50K 39.36
Guatemala >50K 36.67
Haiti <=50K 36.33
Haiti >50K 42.75
Holand-Netherlands <=50K 40.0
Honduras <=50K 34.33
Honduras >50K 60.0
Hong <=50K 39.14
Hong >50K 45.0
Hungary <=50K 31.3
Hungary >50K 50.0
India <=50K 38.23
India >50K 46.48
Iran <=50K 41.44
Iran >50K 47.5
Ireland <=50K 40.95
Ireland >50K 48.0
Italy <=50K 39.62
Italy >50K 45.4
Jamaica <=50K 38.24
Jamaica >50K 41.1
Japan <=50K 41.0
Japan >50K 47.96
Laos <=50K 40.38
Laos >50K 40.0
Mexico <=50K 40.0
Mexico >50K 46

In [94]:
pd.crosstab(
    data["native-country"],
    data["salary"],
    values=data["hours-per-week"],
    aggfunc=np.mean,
).T

native-country,?,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,Germany,Greece,Guatemala,Haiti,Holand-Netherlands,Honduras,Hong,Hungary,India,Iran,Ireland,Italy,Jamaica,Japan,Laos,Mexico,Nicaragua,Outlying-US(Guam-USVI-etc),Peru,Philippines,Poland,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
salary,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
<=50K,40.164760,41.416667,37.914634,37.381818,38.684211,37.985714,42.338235,38.041667,36.030928,40.483333,41.058824,39.139785,41.809524,39.360656,36.325,40.0,34.333333,39.142857,31.3,38.233333,41.44,40.947368,39.625,38.239437,41.000000,40.375,40.003279,36.09375,41.857143,35.068966,38.065693,38.166667,41.939394,38.470588,39.444444,40.15625,33.774194,42.866667,37.058824,38.799127,37.193548,41.6
>50K,45.547945,40.000000,45.641026,38.900000,50.000000,42.440000,47.000000,48.750000,45.000000,44.533333,50.750000,44.977273,50.625000,36.666667,42.750,NaN,60.000000,45.000000,50.0,46.475000,47.50,48.000000,45.400,41.100000,47.958333,40.000,46.575758,37.50000,NaN,40.000000,43.032787,39.000000,41.500000,39.416667,46.666667,51.43750,46.800000,58.333333,40.000000,45.505369,39.200000,49.5


In [77]:
data[data["native-country"]=="Japan"].groupby(["salary","native-country"])["hours-per-week"].agg("mean")

salary  native-country
<=50K   Japan             41.000000
>50K    Japan             47.958333
Name: hours-per-week, dtype: float64

### Bonus

In [ ]:
test = pd.read_csv("shmya_final_version.csv")

In [32]:
test

,date,cutlery,tips,order_price,uid,order_id
0,2022-01-17 06:38:43,3,100,1001,120967,73041443
1,2022-01-23 11:09:22,4,110,309,325757,60500835
2,2022-01-24 19:40:03,1,10,891,986730,43230531
3,2022-01-14 18:01:22,5,70,749,868648,46180789
4,2022-01-19 22:32:43,2,10,469,863523,59266169
...,...,...,...,...,...,...
26052,2022-01-04 05:21:39,4,10,763,182587,85041983
26053,2022-01-18 14:43:12,3,60,790,187031,47894568
26054,2022-01-26 18:08:02,1,90,502,247419,75350869
26055,2022-01-18 05:55:09,2,80,418,474601,79438423


In [46]:
test[(test["order_price"]!=0) & (test["cutlery"]>2)]["tips"].mean()

78.87790616733955

In [48]:
test[(test["order_price"]!=0) & (test["cutlery"]<=2)]["tips"].mean()

51.55533176681213

In [36]:
test[test["cutlery"]>2].order_price.min()

0

In [45]:
test[(test["order_price"]!=0) & (test["cutlery"]>2)].order_price.min()

254

In [50]:
test[(test["order_price"]!=0) & (test["cutlery"]<=2)].order_price.max()

1044

In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26057 entries, 0 to 26056
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         26057 non-null  object
 1   cutlery      26057 non-null  int64 
 2   tips         26057 non-null  int64 
 3   order_price  26057 non-null  int64 
 4   uid          26057 non-null  int64 
 5   order_id     26057 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [54]:
from datetime import datetime

In [76]:
testt = test.query("date < '2022-01-09'")

In [75]:
testt.uid.count()

17221

In [74]:
testt[(testt["order_price"]>800)&(test["cutlery"]>2)].uid

0        120967
18       255228
23       781632
46       638996
52       686259
          ...  
26027    739019
26033    540487
26034    117066
26038    958020
26043    908699
Name: uid, Length: 3529, dtype: int64

In [73]:
testt[(testt["order_price"]>800)&(test["cutlery"]>2)].uid.count()

3529